# 🧩 Maze Scaling Benchmark - Proper Generation

**Using Research-Grade Maze Generation (maze-dataset library)**

This notebook uses the [maze-dataset](https://github.com/understanding-search/maze-dataset) library for high-quality maze generation with controllable difficulty.

## ✨ Key Improvements:
- ✅ **Proper algorithms**: DFS, Wilson's, Prim's (we use DFS for challenging mazes)
- ✅ **Path length filtering**: Control difficulty by minimum solution length
- ✅ **Guaranteed solvable**: No unsolvable mazes
- ✅ **Long planning horizons**: Force 80+ step solutions for differentiation

## Models Compared:
- **Baseline**: Standard Transformer encoder-decoder
- **PoH-HRM**: Pointer-over-Heads with Hierarchical Reasoning Module

## Why Proper Generation Matters:
**Old approach** (random walls): Path length ~16-20 → All models get 100% ❌  
**New approach** (DFS + filtering): Path length 80+ → Clear differentiation ✅

## Recommended Starting Point:
- **Maze Size**: 20×20 (large enough to be challenging)
- **Min Path Length**: 80 (20% of grid size)
- **Training**: 300-500 mazes
- **Goal**: Test if HRM provides advantage on long-horizon planning

---

**Hardware**: Optimized for A100 GPU

**Recent Architecture Fixes (Nov 2025):**
- Fixed head routing to apply to per-head outputs BEFORE projection
- Fixed positional encoding to only apply once (not R times)
- Made HRM H-update gradients configurable (now differentiable by default)
- Added configurable `grad_mode` for memory vs deep supervision tradeoff


## Setup


In [ ]:
# Clone repository and ensure latest fixes
!git clone https://github.com/Eran-BA/PoT.git
%cd PoT
!git pull origin main


Cloning into 'PoT'...
remote: Enumerating objects: 1110, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 1110 (delta 48), reused 51 (delta 25), pack-reused 1035 (from 1)
Receiving objects: 100% (1110/1110), 822.42 KiB | 25.70 MiB/s, done.
Resolving deltas: 100% (589/589), done.
/content/PoT


In [ ]:
# Install dependencies
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers datasets scipy numpy tqdm matplotlib seaborn

# Install maze-dataset library for proper maze generation
!pip install maze-dataset


In [3]:
# Verify GPU
import torch
print(f"GPU Available: {torch.cuda.is_available()}")
print(f"GPU Name: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'N/A'}")
print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB" if torch.cuda.is_available() else "N/A")


GPU Available: True
GPU Name: NVIDIA A100-SXM4-80GB
GPU Memory: 85.17 GB


In [12]:
!git pull origin main

remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 9 (delta 7), reused 9 (delta 7), pack-reused 0 (from 0)
Unpacking objects: 100% (9/9), 1.87 KiB | 478.00 KiB/s, done.
From https://github.com/Eran-BA/PoT
 * branch            main       -> FETCH_HEAD
   9798f26..1fb1efb  main       -> origin/main
Updating 9798f26..1fb1efb
Fast-forward
 experiments/maze_ab_proper_generation.py | 40 ++++++++++++++++++++------------
 experiments/maze_hyperparam_search.py    | 36 +++++++++++++++++++++++++---
 2 files changed, 58 insertions(+), 18 deletions(-)


In [ ]:
!python3 experiments/maze_hyperparam_search.py --quick --epochs 100

In [ ]:
!python experiments/maze_hyperparam_search.py \
  --maze-size 12 \
  --train 1000 \
  --test 100 \
  --epochs 200

In [36]:
!python experiments/analyze_maze_hyperparam_results.py \
  experiments/results/maze_hyperparam_search_12x12.csv


Loading results from: experiments/results/maze_hyperparam_search_12x12.csv

Summary Statistics

Total configurations tested: 61

PoH Accuracy:
  Mean:   64.11%
  Std:    28.04%
  Min:    0.00%
  Max:    93.00%

PoH Optimality:
  Mean:   44.43%
  Std:    37.49%
  Min:    0.00%
  Max:    85.00%

Baseline Accuracy:
  Mean:   0.85%
  Std:    6.66%

Baseline Optimality:
  Mean:   0.69%
  Std:    5.38%


BEST OVERALL CONFIGURATION (weighted: 70% acc + 30% opt)

R=2, T=4, n_heads=2

PoH-HRM Results:
  Accuracy:   93.00%
  Optimality: 85.00%
  Score:      90.60

Baseline Results:
  Accuracy:   0.00%
  Optimality: 0.00%

Improvement:
  Accuracy:   0.0%
  Optimality: 0.0%


Top 5 Configurations by poh_acc

1. R=2, T=4, heads=2
   PoH: Acc=93.00%, Opt=85.00%
   Baseline: Acc=0.00%, Opt=0.00%
   Improvement: Acc=0.0%, Opt=0.0%

2. R=4, T=8, heads=8
   PoH: Acc=93.00%, Opt=84.00%
   Baseline: Acc=0.00%, Opt=0.00%
   Improvement: Acc=0.0%, Opt=0.0%

3. R=8, T=2, heads=2
   PoH: Acc=92.00%, Opt=83.0

In [37]:
from google.colab import files

# Use full path to the CSV file
files.download('/content/PoT/experiments/results/maze_hyperparam_search_12x12.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 🚀 Quick Test (Recommended First)

**Start here!** Test with challenging 20×20 mazes:
- Min path length: 80 (forces long-horizon planning)
- Training: 300 mazes, Test: 50 mazes  
- Epochs: 30
- PoH: R=4, T=4, heads=4

**Expected Runtime on A100**: ~45 minutes  
**Goal**: Test if proper maze generation provides model differentiation


In [39]:
!python experiments/maze_ab_proper_generation.py \
  --maze-size 20 \
  --train 1000 \
  --test 100 \
  --min-path-length 80 \
  --epochs 200 \
  --R 4 --T 8 --heads 8 \
  --seed 42


✓ PoT root: /content/PoT
✓ maze-dataset library available
2025-10-15 01:43:48.344111: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-15 01:43:48.362147: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760492628.383778  309456 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760492628.390372  309456 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1760492628.407235  309456 computation_placer.cc:177] computation placer a

In [40]:
!python experiments/maze_ab_proper_generation.py \
  --maze-size 20 \
  --train 1000 \
  --test 100 \
  --min-path-length 80 \
  --epochs 100 \
  --R 8 --T 12 --heads 8 \
  --seed 42


✓ PoT root: /content/PoT
✓ maze-dataset library available
2025-10-15 02:44:28.317957: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-15 02:44:28.336164: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760496268.358096  324450 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760496268.364765  324450 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1760496268.381617  324450 computation_placer.cc:177] computation placer a

## Medium Test (24×24 Mazes)

**More challenging** with larger mazes:
- Min path length: 120 (longer planning horizon)
- Training: 400 mazes, Test: 80 mazes
- Epochs: 35
- PoH: R=4, T=4, heads=4

**Expected Runtime on A100**: ~70 minutes  
**Goal**: Test if gap increases with maze complexity


In [ ]:
!python experiments/maze_ab_proper_generation.py \
  --maze-size 24 \
  --train 1000 \
  --test 100 \
  --min-path-length 120 \
  --epochs 200 \
  --R 4 --T 8 --heads 8 \
  --seed 42


## Hard Test (30×30 Mazes)

**Maximum challenge** - test extreme long-horizon planning:
- Min path length: 180 (extreme planning horizon)
- Training: 500 mazes, Test: 100 mazes
- Epochs: 40
- PoH: R=4, T=4, heads=4

**Expected Runtime on A100**: ~2 hours  
**Goal**: Test if HRM advantage scales to very large mazes


In [ ]:
!python experiments/maze_ab_proper_generation.py \
  --maze-size 30 \
  --train 500 \
  --test 100 \
  --min-path-length 180 \
  --epochs 40 \
  --R 4 --T 4 --heads 4 \
  --seed 42


## Adjusting Difficulty

If you need to tune maze difficulty based on results, modify `--min-path-length`:

**If both models get >95% accuracy:**
- Mazes are too easy
- Increase `--min-path-length` by 20-40
- Example: Change 80 → 100 or 120

**If both models get <50% accuracy:**
- Mazes are too hard  
- Decrease `--min-path-length` by 20-40
- Example: Change 80 → 60 or 50

**Goal:** Find difficulty where models show clear differentiation (not both 100% or both failing)


In [32]:
# Example: Custom difficulty adjustment
# If 20×20 with min_path=80 is too easy/hard, try:

# Easier (shorter paths):
# !python experiments/maze_ab_proper_generation.py --maze-size 20 --train 300 --test 50 --min-path-length 60 --epochs 30 --R 4 --T 4 --heads 4

# Harder (longer paths):
# !python experiments/maze_ab_proper_generation.py --maze-size 20 --train 300 --test 50 --min-path-length 120 --epochs 30 --R 4 --T 4 --heads 4

# Different size with auto path length (40% of grid):
# !python experiments/maze_ab_proper_generation.py --maze-size 25 --train 400 --test 80 --min-path-length 250 --epochs 35 --R 4 --T 4 --heads 4


## 📊 Interpreting Results

After running the benchmark, look for this in the console output:

**Console Output Format:**
```
Baseline: Acc=XX.X%, Opt=XX.X%
PoH-HRM:  Acc=XX.X%, Opt=XX.X%
```

**Key Metrics:**
- **Accuracy**: Did the model find a path to the goal?
- **Optimality**: Did it find the shortest path?
- **Path Length**: Average solution length in the dataset

**What to Analyze:**
- **Gap size**: How much better (or worse) is PoH-HRM vs Baseline?
- **Trends**: Does the gap change with maze size?
- **Absolute performance**: Are mazes at appropriate difficulty level?


In [33]:
# Check console output above for results
# The script prints a summary like:
#
# ============================================================
# Training: Baseline (Transformer)
# ============================================================
# Parameters: X.XXM
# Epoch 30/30 | Loss: X.XXXX | Acc: XX.XX% | Opt: XX.XX%
#
# Final Performance:
#   Accuracy: XX.XX%
#   Optimality: XX.XX%
#
# ============================================================
# Training: PoH-HRM (R=4, T=4)
# ============================================================
# Parameters: X.XXM
# Epoch 30/30 | Loss: X.XXXX | Acc: XX.XX% | Opt: XX.XX%
#
# Final Performance:
#   Accuracy: XX.XX%
#   Optimality: XX.XX%

print("Check the console output above for complete results!")


Check the console output above for complete results!


In [34]:
# Optional: Save your actual results to a file for later analysis
# After running the test, update this with your real numbers:

results_text = """
Maze A/B Test Results
======================

Test Configuration:
- Maze Size: [YOUR_SIZE]
- Min Path Length: [YOUR_MIN_PATH]
- Training Samples: [YOUR_TRAIN]
- Test Samples: [YOUR_TEST]

Results:
- Baseline:  Acc=[X]%, Opt=[Y]%
- PoH-HRM:   Acc=[X]%, Opt=[Y]%
- Improvement: [calculate difference]

Conclusion: [Your analysis here]
"""

# Uncomment to save:
# with open('maze_results_summary.txt', 'w') as f:
#     f.write(results_text)
# print("Results saved!")


## Why HRM Might Help on Long-Horizon Planning

### The Architecture:

**PoH-HRM** uses a two-timescale recurrent controller:
- **f_L (fast module)**: Updates every step, handles local navigation
- **f_H (slow module)**: Updates every T=4 steps, guides overall strategy

**Baseline** uses flat attention:
- All reasoning happens at one timescale
- No hierarchical decomposition

### The Hypothesis:

On mazes with 80+ step solutions:
- **Baseline**: Must plan entire path in one shot → potentially harder
- **PoH-HRM**: f_H sets high-level direction, f_L executes → potentially easier

This is the hypothesis from the [HRM paper](https://arxiv.org/pdf/2506.21734): hierarchical reasoning should help on tasks requiring long-horizon planning.

### What to Look For:

- **Accuracy gap** favoring PoH-HRM  
- **Gap may increase** with maze size (20→24→30)  
- **Optimality difference** (which model finds shorter paths?)

**Note:** These are predictions - run the experiments to see if they hold!


## Export Results

Download results to include in paper/documentation


In [ ]:
# Download results
from google.colab import files

files.download('experiments/results/maze_scaling_full.json')
files.download('experiments/results/maze_scaling_full.png')
